In [61]:
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
sns.set(rc={'figure.figsize':(12,3)})
pd.set_option('display.max_columns', 50)
from sklearn.metrics import r2_score,confusion_matrix,make_scorer,accuracy_score,classification_report,precision_score,f1_score,recall_score,ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, KFold,cross_validate,StratifiedKFold,cross_val_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,LabelEncoder,OrdinalEncoder
le=LabelEncoder()
oe=OrdinalEncoder()
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm_notebook
df=pd.read_excel('df_.xlsx')
#df.drop(8,axis=0,inplace=True)
df

,S/N,NAME,STATE,Previously in Political office,Marital Status,Educational Qualifications,Political Party,Age at First Entry,influential Figure,Previous Occupation,Votes Won,Office Won (2023)
0,1,Nkeiruka C. Onyejeocha,Abia,Yes,Married,M.Sc.,APC,38,Unknown,Administration/ Managing director,38,unknown
1,2,Lynda Chuba Ikpeazu,Anambra,Yes,Unknown,M.Sc.,PDP,33,Yes,Beauty pageant/ business woman,33,unknown
2,3,Onuh Onyeche Blessing,Benue,Yes,Married,M.Sc.,APC,43,Yes,Government administrator,43,House of Representatives
3,4,Zainab Gimba \n\n,Borno,Yes,Married,Ph.D,APC,47,Unknown,Government administrator,47,House of Representatives
4,5,Ogunlola Omowumi Olubunmi,Ekiti,Yes,Unknown,LLB,APC,54,Yes,Lawyer,54,unknown
5,6,Aishatu Jibril Dukku,Gombe,Yes,Married,B.Sc.,APC,52,Unknown,Academia,52,unknown
6,7,Onanuga Adewunmi Oriyomi \n\n,Ogun,Yes,Unknown,MBA,APC,54,Unknown,Office administration,54,House of Representatives
7,8,Obiageli Orogbu,Anambra,Yes,Unknown,Ph.D,LP,50,Unknown,\nProfessor/ academia\n,50,House of Representatives
8,9,Ireti Kingibe,Federal Capital Territory,Unknown,Married,B.Sc.,LP,68,Yes,Engineer,68,Senate
9,10,Harry Banigo,Rivers,Yes,Married,M.Sc.,PDP,66,Yes,"Medical Doctor, Civil Servant",66,Senate


In [62]:
df['Age at First Entry'].fillna(0,inplace=True)
df['Votes Won'].fillna(0,inplace=True)
df['Votes Won']=df['Age at First Entry'].astype(int)
df['influential Figure'].fillna(value='unknown',inplace=True)
df['Age at First Entry']=df['Age at First Entry'].astype(int)

In [63]:
def summary(df):
    sum = pd.DataFrame(df.dtypes, columns=['dtypes'])
    sum['missing_num'] = df.isna().sum().values*100
    sum['missing%'] = (df.isna().sum().values*100)/len(df)
    sum['uniques'] = df.nunique().values
    sum['count'] = df.count().values
    desc = pd.DataFrame(df.describe().T)
    sum['min'] = desc['min']
    sum['max'] = desc['max']
    sum['mean'] = desc['mean']
    sum['25%'] = desc['25%']
    sum['75%'] = desc['75%']
    return sum
summary(df).style.background_gradient(cmap='Blues')

,dtypes,missing_num,missing%,uniques,count,min,max,mean,25%,75%
S/N,int64,0,0.000000,25,25,1.000000,25.000000,13.000000,7.000000,19.000000
NAME,object,0,0.000000,25,25,nan,nan,nan,nan,nan
STATE,object,0,0.000000,18,25,nan,nan,nan,nan,nan
Previously in Political office,object,0,0.000000,3,25,nan,nan,nan,nan,nan
Marital Status,object,0,0.000000,5,25,nan,nan,nan,nan,nan
Educational Qualifications,object,0,0.000000,10,25,nan,nan,nan,nan,nan
Political Party,object,0,0.000000,5,25,nan,nan,nan,nan,nan
Age at First Entry,int32,0,0.000000,17,25,33.000000,68.000000,50.840000,43.000000,54.000000
influential Figure,object,0,0.000000,3,25,nan,nan,nan,nan,nan
Previous Occupation,object,0,0.000000,17,25,nan,nan,nan,nan,nan


In [64]:
obj=df.select_dtypes('object')
num=df.select_dtypes(exclude='object')

In [65]:
for col in num:
    print(col,f"has {df[col].nunique()}(different samples/unique entry)")
    print(f"Unique values in {[col]} column:{df[col].unique()}")
    print('_-_'*25)

S/N has 25(different samples/unique entry)
Unique values in ['S/N'] column:[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
Age at First Entry has 17(different samples/unique entry)
Unique values in ['Age at First Entry'] column:[38 33 43 47 54 52 50 68 66 55 51 67 42 40 49 44 53]
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
Votes Won has 17(different samples/unique entry)
Unique values in ['Votes Won'] column:[38 33 43 47 54 52 50 68 66 55 51 67 42 40 49 44 53]
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_


In [66]:
for col in num:
    print(col, f"frequency of times each sample it occurs \n {df[col].value_counts()}" ,sep=':')
    print('_-_'*25)

S/N:frequency of times each sample it occurs 
 1     1
14    1
24    1
23    1
22    1
21    1
20    1
19    1
18    1
17    1
16    1
15    1
13    1
2     1
12    1
11    1
10    1
9     1
8     1
7     1
6     1
5     1
4     1
3     1
25    1
Name: S/N, dtype: int64
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
Age at First Entry:frequency of times each sample it occurs 
 66    2
47    2
54    2
52    2
50    2
68    2
40    2
42    2
67    1
44    1
49    1
38    1
51    1
55    1
33    1
43    1
53    1
Name: Age at First Entry, dtype: int64
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
Votes Won:frequency of times each sample it occurs 
 66    2
47    2
54    2
52    2
50    2
68    2
40    2
42    2
67    1
44    1
49    1
38    1
51    1
55    1
33    1
43    1
53    1
Name: Votes Won, dtype: int64
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_


In [67]:
for col in obj:
    print(col,f"has {df[col].nunique()}(different samples/unique entry)")
    print(f"Unique values in {[col]} column:{df[col].unique()}")
    print('_-_'*25)

NAME has 25(different samples/unique entry)
Unique values in ['NAME'] column:['Nkeiruka C. Onyejeocha' 'Lynda Chuba Ikpeazu ' 'Onuh Onyeche Blessing '
 'Zainab Gimba \n\n' 'Ogunlola Omowumi Olubunmi ' 'Aishatu Jibril Dukku '
 'Onanuga Adewunmi Oriyomi \n\n' 'Obiageli Orogbu ' 'Ireti Kingibe'
 ' Harry Banigo' ' Idiat Adebule' ' Marie Ebikake' 'Nnabuife Chinwe Clara'
 'Gwacham Maureen Chime' 'Regina Akume' 'Ibori-Suenu Erhiatake'
 'Fatima Talba' 'Beni Butmak Lar' 'Chigeru Blessing Amadi' 'Goodhead Boma'
 'Khadija Bukar Abba Ibrahim' 'Onuoha Miriam Odinaka' 'Kafilat Ogbara'
 'Tolulope Akande-Sadipe' 'Natasha Akpoti-Uduaghan']
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
STATE has 18(different samples/unique entry)
Unique values in ['STATE'] column:['Abia' 'Anambra' 'Benue' 'Borno' 'Ekiti' 'Gombe' 'Ogun' 'Anambra  '
 'Federal Capital Territory' 'Rivers' 'Lagos ' 'Bayelsa' 'Delta' 'Yobe'
 'Plateau' 'Imo' 'Oyo' 'Kogi']
_-__-__-__-__-__-__-__-__-__-__-__-__-__-_

In [68]:
for col in obj:
    print(col, f"frequency of times each sample it occurs \n {df[col].value_counts()}" ,sep=':')
    print('_-_'*25)

NAME:frequency of times each sample it occurs 
 Nkeiruka C. Onyejeocha           1
Gwacham Maureen Chime            1
Tolulope Akande-Sadipe           1
Kafilat Ogbara                   1
Onuoha Miriam Odinaka            1
Khadija Bukar Abba Ibrahim       1
Goodhead Boma                    1
Chigeru Blessing Amadi           1
Beni Butmak Lar                  1
Fatima Talba                     1
Ibori-Suenu Erhiatake            1
Regina Akume                     1
Nnabuife Chinwe Clara            1
Lynda Chuba Ikpeazu              1
 Marie Ebikake                   1
 Idiat Adebule                   1
 Harry Banigo                    1
Ireti Kingibe                    1
Obiageli Orogbu                  1
Onanuga Adewunmi Oriyomi \n\n    1
Aishatu Jibril Dukku             1
Ogunlola Omowumi Olubunmi        1
Zainab Gimba \n\n                1
Onuh Onyeche Blessing            1
Natasha Akpoti-Uduaghan          1
Name: NAME, dtype: int64
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__

In [69]:
df.columns

Index(['S/N', 'NAME', 'STATE', 'Previously in Political office',
       'Marital Status', 'Educational Qualifications ', 'Political Party',
       'Age at First Entry', 'influential Figure', 'Previous Occupation',
       'Votes Won', 'Office Won (2023)'],
      dtype='object')

In [70]:
target_column = 'Votes Won'
def generate_plots(df, target_column):
    for column in df.columns:
        if column != target_column:
            if df[column].dtype == 'object':  # If the column is categorical
                fig = px.histogram(df, x=column, color=target_column, barmode='group',
                    title=f'Relationship between {column} and {target_column}',
                    labels={column: f'{column} Categories', target_column: 'Votes Won'})
            else:  # If the column is numerical, show box plot
                fig = px.box(df, x=target_column, y=column, title=f'Relationship between {column} and {target_column}')
            fig.show()
generate_plots(df, target_column)

In [71]:
df.columns

Index(['S/N', 'NAME', 'STATE', 'Previously in Political office',
       'Marital Status', 'Educational Qualifications ', 'Political Party',
       'Age at First Entry', 'influential Figure', 'Previous Occupation',
       'Votes Won', 'Office Won (2023)'],
      dtype='object')

In [72]:
def generate_categorical_comparison(df):
    categorical_columns = ['STATE', 'Previously in Political office','Marital Status', 'Educational Qualifications ', 'Political Party',
       'Age at First Entry', 'influential Figure', 'Previous Occupation',
       'Votes Won', 'Office Won (2023)']
    for column in categorical_columns:
        # Generate bar plot
        fig = px.bar(df, x='Votes Won', y='Office Won (2023)', color=column, 
                     title=f'Comparison of Votes Won by {column}', 
                     hover_data=['Votes Won', 'Office Won (2023)'],  # Show names and percentage won in hover
                     labels={'Office Won (2023)': 'political ranking', 'Votes Won': 'votes won by candidate'})
        fig.show()
generate_categorical_comparison(df)


In [12]:
fig = px.bar(df, x='Previous Positions', y='Percentage of votes won', color='Age at First Entry', barmode='group',
             title='Relationship Between Two Categorical Columns')
fig.show()

In [13]:
grouping = df.groupby('Previous Positions')['Percentage of votes won'].sum().reset_index()
px.bar(y='Percentage of votes won', x='Previous Positions', data_frame=grouping)

# multivariate analysis 

In [67]:
grouping = df.groupby('country')['total_cost'].sum().reset_index()
px.bar(y='country', x='total_cost', data_frame=grouping)

In [68]:
grouping = df.groupby('age_group')['total_cost'].sum().reset_index()
px.bar(y='age_group', x='total_cost', data_frame=grouping)

In [69]:
grouping = df.groupby('travel_with')['total_cost'].sum().reset_index()
px.bar(y='travel_with', x='total_cost', data_frame=grouping)

In [70]:
grouping = df.groupby('info_source')['total_cost'].sum().reset_index()
px.bar(y='info_source', x='total_cost', data_frame=grouping)

In [71]:
grouping = df.groupby('main_activity')['total_cost'].sum().reset_index()
px.bar(y='main_activity', x='total_cost', data_frame=grouping)

In [72]:
grouping = df.groupby('purpose')['total_cost'].sum().reset_index()
px.bar(y='purpose', x='total_cost', data_frame=grouping)

In [73]:
grouping = df.groupby('most_impressing')['total_cost'].sum().reset_index()
px.bar(y='most_impressing', x='total_cost', data_frame=grouping)

In [74]:
grouping = df.groupby('payment_mode')['total_cost'].sum().reset_index()
px.bar(y='payment_mode', x='total_cost', data_frame=grouping)

# regressional analysis

In [75]:
px.scatter(x='total_cost',y='night_mainland',data_frame=df)

In [76]:
px.scatter(x='total_cost',y='night_zanzibar',data_frame=df)

In [77]:
obj=df.select_dtypes('object')
num=df.select_dtypes(exclude='object')

In [78]:
for i in obj:
    print(obj[i].unique())

['tour_0' 'tour_10' 'tour_1000' ... 'tour_992' 'tour_996' 'tour_998']
['SWIZERLAND' 'UNITED KINGDOM' 'CHINA' 'SOUTH AFRICA'
 'UNITED STATES OF AMERICA' 'NIGERIA' 'INDIA' 'BRAZIL' 'CANADA' 'MALT'
 'MOZAMBIQUE' 'RWANDA' 'AUSTRIA' 'MYANMAR' 'GERMANY' 'KENYA' 'ALGERIA'
 'IRELAND' 'DENMARK' 'SPAIN' 'FRANCE' 'ITALY' 'EGYPT' 'QATAR' 'MALAWI'
 'JAPAN' 'SWEDEN' 'NETHERLANDS' 'UAE' 'UGANDA' 'AUSTRALIA' 'YEMEN'
 'NEW ZEALAND' 'BELGIUM' 'NORWAY' 'ZIMBABWE' 'ZAMBIA' 'CONGO' 'BURGARIA'
 'PAKISTAN' 'GREECE' 'MAURITIUS' 'DRC' 'OMAN' 'PORTUGAL' 'KOREA'
 'SWAZILAND' 'TUNISIA' 'KUWAIT' 'DOMINICA' 'ISRAEL' 'FINLAND'
 'CZECH REPUBLIC' 'UKRAIN' 'ETHIOPIA' 'BURUNDI' 'SCOTLAND' 'RUSSIA'
 'GHANA' 'NIGER' 'MALAYSIA' 'COLOMBIA' 'LUXEMBOURG' 'NEPAL' 'POLAND'
 'SINGAPORE' 'LITHUANIA' 'HUNGARY' 'INDONESIA' 'TURKEY' 'TRINIDAD TOBACCO'
 'IRAQ' 'SLOVENIA' 'UNITED ARAB EMIRATES' 'COMORO' 'SRI LANKA' 'IRAN'
 'MONTENEGRO' 'ANGOLA' 'LEBANON' 'SLOVAKIA' 'ROMANIA' 'MEXICO' 'LATVIA'
 'CROATIA' 'CAPE VERDE' 'SUDAN' 'COSTARICA

In [79]:
obj.columns

Index(['ID', 'country', 'age_group', 'travel_with', 'purpose', 'main_activity',
       'info_source', 'tour_arrangement', 'package_transport_int',
       'package_accomodation', 'package_food', 'package_transport_tz',
       'package_sightseeing', 'package_guided_tour', 'package_insurance',
       'payment_mode', 'first_trip_tz', 'most_impressing'],
      dtype='object')

# if you like this and want my services im open 24/7 my contacts are: +2348109725406(whatsapp\call),
emails: agboolayusf2018@gmail.com,yusuf.m1902547@futminna.edu.ng
    linkedin: tajudeen yusuf agboola